# 🫀 ECG Cross-Attention Auto Experiment

MIT-BIH Arrhythmia Database를 사용한 ECG 부정맥 분류 **자동 실험**

## 실험 목록
| 모델 | * (DS1 전체 22명) | @ (DS1-1/DS1-2 split) |
|------|-------------------|----------------------|
| **A0** (Baseline, Dense) | A0* | A0@ |
| **A1** (Naive Concat, Dense) | A1* | A1@ |
| **A2** (Cross Attention, Dense) | A2* | A2@ |
| **B0** (Baseline, No Dense) | B0* | B0@ |
| **B1** (Naive Concat, No Dense) | B1* | B1@ |
| **B2** (Cross Attention, No Dense) | B2* | B2@ |

**분류 클래스 (AAMI 4-class)**: N, S, V, F


## 1️⃣ 환경 설정


In [ ]:
# GPU 확인
!nvidia-smi


In [ ]:
# 필요한 패키지 설치
!pip install -q wfdb openpyxl neurokit2


In [ ]:
# GitHub에서 프로젝트 clone
!git clone https://github.com/h4xryu/ECG_CrossAttention.git
%cd ECG_CrossAttention


## 2️⃣ Google Drive 마운트 & 데이터 연결


In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive 마운트 완료!")



In [ ]:
import os

# Google Drive의 데이터 경로
DRIVE_DATA_PATH = '/content/drive/MyDrive/ECG_Data/mit-bih-arrhythmia-database-1.0.0/'

# 프로젝트 데이터 폴더 생성 및 심볼릭 링크
os.makedirs('./data', exist_ok=True)

# 심볼릭 링크 생성 (이미 있으면 삭제 후 재생성)
link_path = './data/mit-bih-arrhythmia-database-1.0.0'
if os.path.islink(link_path):
    os.unlink(link_path)
elif os.path.exists(link_path):
    import shutil
    shutil.rmtree(link_path)

os.symlink(DRIVE_DATA_PATH, link_path)

# 데이터 확인
print("📂 데이터 경로:", link_path)
print("\n파일 목록:")
!ls -la {link_path} | head -20

# 레코드 수 확인
dat_files = [f for f in os.listdir(link_path) if f.endswith('.dat')]
print(f"\n✅ 총 {len(dat_files)}개 레코드 확인됨")



## 3️⃣ 자동 실험 설정 확인


In [ ]:
# main_autoexp.py 설정 확인
from main_autoexp import (
    EXPERIMENTS, EPOCHS, BATCH_SIZE, LR, CLASSES,
    DS1_FULL, DS1_TRAIN_SPLIT, DS1_VALID_SPLIT, DS2_TEST,
    MODEL_CONFIG, OUTPUT_PATH
)

print("=" * 60)
print("🔬 자동 실험 설정")
print("=" * 60)
print(f"총 실험 수: {len(EXPERIMENTS)}")
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Learning Rate: {LR}")
print(f"Classes: {CLASSES}")
print(f"Output: {OUTPUT_PATH}")

print("\n📋 실험 목록:")
for exp_name, model_type, data_config in EXPERIMENTS:
    config_str = "DS1 전체 22명" if data_config == 'star' else "DS1-1(17)/DS1-2(5) split"
    print(f"  {exp_name:6s} | {model_type:25s} | {config_str}")

print(f"\n📂 데이터:")
print(f"  DS1 Full ({len(DS1_FULL)}명): {DS1_FULL[:5]}...")
print(f"  DS1-1 Train ({len(DS1_TRAIN_SPLIT)}명): {DS1_TRAIN_SPLIT[:5]}...")
print(f"  DS1-2 Valid ({len(DS1_VALID_SPLIT)}명): {DS1_VALID_SPLIT}")
print(f"  DS2 Test ({len(DS2_TEST)}명): {DS2_TEST[:5]}...")


In [ ]:
# [선택] 빠른 테스트를 위해 에폭 수 변경
# 아래 주석을 해제하면 5 에폭으로 빠르게 테스트 가능

# !sed -i 's/EPOCHS = 50/EPOCHS = 5/' main_autoexp.py
# !grep "^EPOCHS" main_autoexp.py


## 4️⃣ 자동 실험 실행 (12개 실험)


In [ ]:
# 🚀 자동 실험 실행 (12개 실험 x 50 에폭)
# 예상 소요 시간: GPU에 따라 다름 (T4 기준 약 6-8시간)
!python main_autoexp.py


In [ ]:
# 결과 폴더 확인
!ls -la auto_results/


In [ ]:
# 전체 결과 요약 Excel 확인
import glob
import pandas as pd

summary_files = sorted(glob.glob('auto_results/Results_Summary_*.xlsx'))
if summary_files:
    latest_summary = summary_files[-1]
    print(f"📊 결과 요약 파일: {latest_summary}")
    
    df = pd.read_excel(latest_summary, sheet_name='Performance Metrics', header=None)
    print("\n" + "="*80)
    print("Performance Metrics Summary")
    print("="*80)
    print(df.to_string())
else:
    print("❌ 결과 파일이 없습니다. 먼저 실험을 실행하세요.")


In [ ]:
# 개별 실험 결과 확인
import pandas as pd
import glob

exp_dirs = sorted(glob.glob('auto_results/A*') + glob.glob('auto_results/B*'))
print(f"📂 총 {len(exp_dirs)}개 실험 폴더")

for exp_dir in exp_dirs[:3]:  # 처음 3개만 표시
    xlsx_files = glob.glob(f'{exp_dir}/*.xlsx')
    for xlsx in xlsx_files:
        print(f"\n{'='*60}")
        print(f"📊 {xlsx}")
        print("=" * 60)
        
        df_macro = pd.read_excel(xlsx, sheet_name='Macro')
        print(df_macro.to_string(index=False))


In [ ]:
# Confusion Matrix 이미지 표시 (모든 실험)
from IPython.display import Image, display
import glob

exp_dirs = sorted(glob.glob('auto_results/A*') + glob.glob('auto_results/B*'))

for exp_dir in exp_dirs:
    png_files = glob.glob(f'{exp_dir}/*.png')
    for png in png_files:
        print(f"\n📊 {png}")
        display(Image(filename=png, width=400))


## 5️⃣ 결과를 Google Drive에 저장


In [ ]:
# auto_results 폴더 전체를 Google Drive로 복사
import shutil
import os

src_path = './auto_results'
dest_path = '/content/drive/MyDrive/ECG_AutoExp_Results'

if os.path.exists(src_path):
    # 기존 폴더 삭제 후 복사
    if os.path.exists(dest_path):
        shutil.rmtree(dest_path)
    shutil.copytree(src_path, dest_path)
    print(f"✅ 전체 결과 저장 완료: {dest_path}")
    !ls -la {dest_path}
else:
    print("❌ auto_results 폴더가 없습니다. 먼저 실험을 실행하세요.")


In [ ]:
# 결과 요약 Excel만 다운로드
import glob
from google.colab import files

summary_files = glob.glob('auto_results/Results_Summary_*.xlsx')
if summary_files:
    for f in summary_files:
        files.download(f)
        print(f"📥 다운로드: {f}")
else:
    print("❌ 결과 파일이 없습니다.")


---
## 📝 빠른 테스트 (5 에폭, 선택된 실험만)


In [ ]:
# 빠른 테스트용 설정 변경 (5 에폭, A2*와 B2*만)
!sed -i 's/EPOCHS = 50/EPOCHS = 5/' main_autoexp.py

# 실험 목록 축소 (A2*, B2* 만)
!sed -i "s/EXPERIMENTS = \\[/EXPERIMENTS = [\\n    ('A2*', 'cross_attention', 'star'),\\n    ('B2*', 'cross_attention_B', 'star'),\\n] # QUICK_TEST\\nEXPERIMENTS_FULL = [/" main_autoexp.py

# 변경 확인
!grep -E "^EPOCHS|EXPERIMENTS" main_autoexp.py | head -5


In [ ]:
# 빠른 테스트 실행 (2개 실험 x 5 에폭)
!python main_autoexp.py
